In [1]:
# # Clone xvr repo and move into the directory
# !git clone https://github.com/eigenvivek/xvr.git
# %cd xvr

# # Install PyTorch for CUDA 11.8 (Colab T4 GPU)
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install xvr and dependencies
# !pip install git+https://github.com/eigenvivek/xvr.git
# !pip install wandb nibabel torchio tqdm


In [2]:
import torch
import torchvision
import torchaudio
import nibabel as nib
import torchio
import tqdm
import wandb

# Check CUDA availability and GPU info
print("Torch CUDA available:", torch.cuda.is_available())
print("Torch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)

# Check xvr import
try:
    import xvr
    print("✅ xvr imported successfully")
except ImportError as e:
    print("❌ xvr import failed:", e)


Torch CUDA available: True
Torch version: 2.7.1+cu118
GPU Name: NVIDIA RTX 6000 Ada Generation
CUDA Version: 11.8
✅ xvr imported successfully


In [3]:
import os

# Set a value
os.environ["WANDB_API_KEY"] = "524773117955f64051f5e2bd2ecef2642a677a34"

In [4]:
# Fix wandb login inside xvr training script (do this only once)
# !sed -i 's/wandb.login(key=os.environ\["WANDB_API_KEY"\])/wandb.login()/' /usr/local/lib/python3.*/dist-packages/xvr/commands/train.py

In [5]:
# import sys
# import os

# # Add the path to xvr/src so you can import xvr.*
# sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))


In [6]:
import sys
import os

# Step 1: Add xvr/src to Python path so imports work
sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))

# Step 2: Patch wandb.login line in local train.py
# train_py = os.path.join(os.getcwd(), 'xvr', 'src', 'xvr', 'commands', 'train.py')
finetune_py = os.path.join(os.getcwd(), 'src', 'xvr', 'commands', 'finetune.py')

if os.path.exists(finetune_py):
    with open(finetune_py, 'r') as f:
        lines = f.readlines()

    with open(finetune_py, 'w') as f:
        for line in lines:
            f.write(line.replace(
                'wandb.login(key=os.environ["WANDB_API_KEY"])',
                'wandb.login()'
            ))

    print(f"✅ Patched wandb.login() in: {finetune_py}")
else:
    print("❌ finetune.py not found. Check the path.")


✅ Patched wandb.login() in: c:\Users\aksha\Robossis\xvr\src\xvr\commands\finetune.py


In [11]:
import subprocess

# Define input/output paths and pretrained checkpoint
input_path = "SE000002.nii"                     # Input CT volume
output_path = "output_current-model_finetuned_0.007"                # Output directory for finetuned model
ckpt_path = "output_trained\checkpoints\RXVR1.2_training_best.pth"           # Path to pretrained checkpoint

command = [
    "xvr", "finetune",
    "--inpath", input_path,
    "--outpath", output_path,
    "--ckptpath", ckpt_path,
    "--lr", "0.007",                            # Optional: learning rate
    "--batch_size", "100",
    "--n_epochs", "25",
    "--n_batches_per_epoch", "50",
    "--rescale", "1.0",
    "--project", "RXVR1.2",                     # Optional: wandb project name
    "--name", "RXVR1.2_Finetuned_05"
]

# Force UTF-8 decoding to avoid CP1252 issues
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    encoding="utf-8",         # ✅ Force UTF-8 decoding
    errors="replace"          # ✅ Replace problematic characters instead of crashing
)

for line in process.stdout:
    print(line, end='')  # Stream output

wandb: Currently logged in as: sharan12 (sharan12-rowan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in c:\Users\aksha\Robossis\xvr\wandb\run-20250811_135214-etczflx2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run RXVR1.2_Finetuned_05
wandb:  View project at https://wandb.ai/sharan12-rowan-university/RXVR1.2
wandb:  View run at https://wandb.ai/sharan12-rowan-university/RXVR1.2/runs/etczflx2
128

Epoch 0: 100%|██████████| 50/50 [00:47<00:00,  1.06it/s]

Epoch 1: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

Epoch 2: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

Epoch 3: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

Epoch 4: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

Epoch 5: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]

Epoch 6: 100%|██████████| 50/50 [00:45<00:00,  1.09it/s]

Epoch 7: 100%|██████████| 50/50 [00:45<00:00,  1.09it/s]



In [ ]:
# process.kill()
# print("✅ Training process terminated.")

# import torch
# import gc

# # Garbage collect unreferenced objects
# gc.collect()

# # Clear unused memory from PyTorch's CUDA allocator
# torch.cuda.empty_cache()

# print("✅ GPU cache cleared.")


In [ ]:
# !xvr train --help

In [ ]:
# !xvr animate \
#   --inpath "/content/drive/MyDrive/SE000002.nii" \
#   --checkpoint "/content/drive/MyDrive/Trained Registration model/xvr_0125.pth" \
#   --outpath "/content/drive/MyDrive/Trained Registration model" \
#   --height 128 --delx 0.5